In [1]:
import dask.dataframe as dd
from nltk.corpus import stopwords
import nltk
import re
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from dask.diagnostics import ProgressBar
%matplotlib inline
import seaborn as sns
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess
from src.utils.helpers_news_movies import *
from src.utils.movies_genres_utils import *

ModuleNotFoundError: No module named 'src.utils.helpers_news_movies'

## Topic modeling  for news

The goal of this part is to test other topic modeling methods to find themes in news papers. We'll test LDA and LSI methods. We'll also try try to find a title for each theme.

In [ ]:
# Load stopwords
nltk.download('stopwords')
stop_words_ = set(stopwords.words('english'))

In [ ]:
ddf = dd.read_parquet('data/nyt_data.parquet')

In [ ]:
# Filter dataset for a particular year
filtered_ddf = ddf[(ddf['year'] >= 1968) & (ddf['year'] <= 1968)]

filtered_ddf = filtered_ddf.reset_index(drop=True)

In [ ]:
# Combine the text and process it
filtered_ddf['combined_text'] = filtered_ddf['title'] + filtered_ddf['excerpt']
filtered_ddf['processed_text'] = filtered_ddf['combined_text'].apply(preprocess_text, meta=('processed_text', 'str'), args= ('stop_words_',))

## Topic modeling by LDA

In [ ]:
# Create the TF matrix
stop_words = stopwords.words('english')
vectorizer = CountVectorizer(max_df=0.90, min_df=2, stop_words=stop_words, token_pattern=r'\b[a-zA-Z]+\b')

processed_text = filtered_ddf['processed_text'].compute() 
tf_matrix = vectorizer.fit_transform(processed_text)

In [ ]:
# Create and fit the model
lda = LatentDirichletAllocation(n_components=30, random_state=0)
lda.fit(tf_matrix)

In [ ]:
# Find and print the terms for each topic
topics_lda = {}
words = vectorizer.get_feature_names_out()
for idx, topic in enumerate(lda.components_):

    top_words = [words[i] for i in topic.argsort()[-30:]]
    topics_lda[f"Theme {idx+1}"] = top_words

for theme, words in topics_lda.items():
    print(f"{theme}: {' '.join(words)}")

## Topic modeling by LSI

In [ ]:
# Create the TF-IDF matrix
tf_idf_vectorizer = TfidfVectorizer(max_features=5000)  
tf_idf_matrix = tf_idf_vectorizer.fit_transform(filtered_ddf['processed_text'])

In [ ]:
# Create and fit the model
svd_model = TruncatedSVD(n_components=30, random_state=42)
svd_model.fit_transform(tf_idf_matrix)

In [ ]:
# Find and print the terms for each topic
topics_lsi = {}
terms = tf_idf_vectorizer.get_feature_names_out()
for i, topic in enumerate(svd_model.components_):
    top_terms = [terms[j] for j in topic.argsort()[-20:]]
    topics_lsi[f"Topic {i+1}"] = top_terms

for topic, terms in topics_lsi.items():
    print(f"{topic}: {' '.join(terms)}")

We tried to find topics for the year 1968. It seems to work since we recognized some hot topics of society from this year, such as the Vietnam war or students' protests.

## Find title for topic

In [ ]:
# Load a model already trained to encode words
model = KeyedVectors.load_word2vec_format("data/word2vec.bin", binary=True)

In [ ]:
# Find a word that summarizes the themes found above
def find_theme(list_words, model):
    vectors = []
    # Calculate the average vector for the theme
    for w in list_words:
        if w in model:
            vectors.append(model[w])
    if vectors:
        avg_vec = sum(vectors) / len(vectors)
    else:
        avg_vec = None
        
    # Find the closest word
    closest_words = model.most_similar(positive=[avg_vec], topn=1)

    closest_w, sim_max = closest_words[0] 
    return closest_w

In [ ]:
# Print the topics found
for t, ws in topics_lsi.items():
    print(t)
    print(f"Terms {ws}")
    theme = find_theme(ws, model)
    print(f"Theme word : {theme}")

The results are coherent but do not seem sufficient for further analysis. We will maybe have to find n-grams to better define the themes or do it manually.

## Compare Topics in News and in Movies

This part shows an example of how we could explore similarities in societal issues between news articles and movies plots. Once we have our topics of interests, we can do semantic search with a sentence-transformer model and compare their presence over time. We will also do more advanced statistical analysis, which we haven't done yet in this part.

## Load NYT dataset

In [ ]:
# Select a time period and compute
with ProgressBar():
    filtered_ddf = ddf[(ddf['year'] >= 1950) & (ddf['year'] <= 2000)].compute()

filtered_ddf = dd.from_pandas(filtered_ddf, npartitions=100)

# Select only a tiny fraction because the data is too large and a small percentage is sufficient for our analyses
filtered_ddf = filtered_ddf.sample(frac=0.01, random_state=42).reset_index(drop=True)


In [ ]:
# Create a combined text field for further analysis
filtered_ddf['combined_text'] = filtered_ddf['title'] + filtered_ddf['excerpt']

In [ ]:
col_names = ['Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Movie release date','Movie box office revenue', 'Movie runtime', 'Movie languages','Movie countries', 'Movie genres']
df_movies = pd.read_csv('data/MovieSummaries/movie.metadata.tsv', sep='\t', header=None, names=col_names)

In [ ]:
# Reformate dates
df_movies['Movie release date'] = pd.to_datetime(df_movies['Movie release date'], errors='coerce', format='%Y-%m-%d')

df_movies['Movie release date'] = df_movies['Movie release date'].fillna(
    pd.to_datetime(df_movies['Movie release date'].astype(str) + '-01-01', errors='coerce')
)

df_movies['Release Year'] = df_movies['Movie release date'].dt.year

In [ ]:
df_movies.head()

In [ ]:
# Select period of time
start_date = 1950
end_date = 2000

df_movies = df_movies[(df_movies['Release Year'] >= start_date) & (df_movies['Release Year'] < end_date)]

In [ ]:
# Create lists
ids = df_movies["Wikipedia movie ID"].to_list()
movies_dict = load_plot_summaries("data/MovieSummaries/plot_summaries.txt")
movies_dict = {k: movies_dict[k] for k in movies_dict if k in ids}
movie_descriptions = list(movies_dict.values())
movie_ids = list(movies_dict.keys())

## Launch Model for movies topic 1

We use a model from sentence-transformers to do semantic search. We will try the methods for 2 different topics.

In [ ]:
# Load the model
model_miniLM = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# Encode the query
query_embedding = model_miniLM.encode(["women's rights"])

In [ ]:
# Encode the movie descriptions
movie_embeddings = model_miniLM.encode(movie_descriptions, batch_size=64)

In [ ]:
# Calculate similarity with the query
similarities = cosine_similarity(query_embedding, movie_embeddings).flatten()
ids = [movie_ids[i] for i, sim in enumerate(similarities) if sim > 0.2]

In [ ]:
df_movies_new = df_movies[df_movies['Wikipedia movie ID'].isin(ids)]
df_movies_new.head()

## Launch model for news topic 1

In [ ]:
# Encode the text
with ProgressBar():
    news = filtered_ddf['combined_text'].compute().tolist()  # Utilisation de compute() pour Dask
news_embeddings = model_miniLM.encode(news, batch_size=64)

In [ ]:
filtered_ddf_pandas = filtered_ddf.compute()

In [ ]:
# Calculate similarity with the query
similarities = cosine_similarity(query_embedding, news_embeddings).flatten()
similar_entries_indices = np.where(similarities >= 0.3)[0]
filtered_similar_ddf = filtered_ddf_pandas.iloc[similar_entries_indices].reset_index(drop=True)

## Plot evolution

In [ ]:
# Calculate the relative presence of the topic across the years
count_per_year = df_movies_new.groupby('Release Year').size()
total_count_per_year = df_movies.groupby('Release Year').size()

relative_count_per_year = count_per_year / total_count_per_year

# Create the line plot
relative_count_per_year.plot(kind='line', figsize=(10, 6), title="Number of Movies per Year")
plt.xlabel("Year")
plt.ylabel("Number of Movies")
plt.show()


In [ ]:
# Calculate the relative presence of the topic accross the years
count_per_year_similar = filtered_similar_ddf.groupby('year').size()
total_count_per_year_similar = filtered_ddf_pandas.groupby('year').size()

relative_count_per_year_similar = count_per_year_similar / total_count_per_year_similar

relative_count_per_year_similar = relative_count_per_year_similar.dropna()

# Line plot
relative_count_per_year_similar.plot(kind='line', figsize=(10, 6), title="Relative Number of news articles per Year")
plt.xlabel("Year")
plt.ylabel("Number of articles")
plt.show()

The tendencies need to be examined in details to draw conclusions. At first glance, we see that there is no particular tendency for women's rights presence in movies while in news articles it tends to increase.

## Launch model for news topic 2

In [ ]:
# Calculate similarity for the new topic
query_embedding_env = model.encode(["environmental issues"])

similarities_env = cosine_similarity(query_embedding_env, movie_embeddings).flatten()
ids_env = [movie_ids[i] for i, sim in enumerate(similarities_env) if sim > 0.2]

df_movies_new_env = df_movies[df_movies['Wikipedia movie ID'].isin(ids_env)]
df_movies_new_env

In [ ]:
# Calculate the relative presence of the topic accross the years
count_per_year = df_movies_new_env.groupby('Release Year').size()

total_count_per_year = df_movies.groupby('Release Year').size()

relative_count_per_year = count_per_year / total_count_per_year

relative_count_per_year = relative_count_per_year.dropna()

# Create the line plot
relative_count_per_year.plot(kind='line', figsize=(10, 6), title="Relative Number of Movies per Year")
plt.xlabel("Year")
plt.ylabel("Number of Movies")

# Generate a linear regression line to the data
x = relative_count_per_year.index.astype(int)  # Ensure 'Year' is in integer format
y = relative_count_per_year.values
a, b = np.polyfit(x, y, 1)  # 1 indicates a linear fit (degree 1 polynomial)

regression_line = a * x + b

plt.plot(x, regression_line, color='red', label='Tendency')
plt.legend()

plt.show()

In [ ]:
# Calculate similarity with new query
similarities_env_news = cosine_similarity(query_embedding_env, news_embeddings).flatten()
similar_entries_indices_env_news = np.where(similarities_env_news >= 0.4)[0]
filtered_similar_ddf_env_news = filtered_ddf_pandas.iloc[similar_entries_indices_env_news].reset_index(drop=True)

In [ ]:
# Calculate the relative presence of the topic accross the years
count_per_year_similar = filtered_similar_ddf_env_news.groupby('year').size()

relative_count_per_year_similar = count_per_year_similar / total_count_per_year_similar

relative_count_per_year_similar = relative_count_per_year_similar.dropna()

# Create a line plot
relative_count_per_year_similar.plot(kind='line', figsize=(10, 6), title="Relative number of articles per year")
plt.xlabel("Year")
plt.ylabel("Number of news articles")

# Generate a linear regression line to the data
x = relative_count_per_year_similar.index.astype(int)  
y = relative_count_per_year_similar.values
a, b = np.polyfit(x, y, 1) 

regression_line = a * x + b

plt.plot(x, regression_line, color='red', label='Tendency')

plt.show()

In this case, we observe that the relative presence of "environmental issues" tends to increase over time. We would need more in-depth analysis to draw conclusions.

# Initial results about movies dataset

In [ ]:
# Load preprocessed data
data_folder = 'data/MovieSummaries/'
df_metadata = pd.read_csv(data_folder + 'movies_metadata_cleaned.csv')

df_metadata

In [ ]:
# Identify NaNs in the 'Grouped_genres' and remove them
problematic_values = []
for index, value in df_metadata['Grouped_genres'].items(): 
    if isinstance(value, str):
        try:
            ast.literal_eval(value)
        except (ValueError, SyntaxError):
            problematic_values.append((index, value))

            
df_metadata = df_metadata.drop([index for index, _ in problematic_values])

## Analysis of genres across the world throughout the decades

In [ ]:
# Count of Genres per decade

# Expand the genres
df_metadata['Grouped_genres'] = df_metadata['Grouped_genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_expanded = df_metadata.explode('Grouped_genres')

# Group by 'Decade' and 'Movie_genres' and count occurrences
genre_distribution = df_expanded.groupby(['Decade', 'Grouped_genres']).size().reset_index(name="Count")

# Sort 'Decade' and 'Count' in descending order
genre_distribution = genre_distribution.sort_values(by=['Decade', 'Count'], ascending=[True, False])

In [ ]:
plot_top_genres(5, genre_distribution)

### Observations

We can observe the evolution of movie genres across the decades. Some genres, like Silent Films, disappeared from the top 5, while others emerged over time. Certain genres, such as Romance and Action/Adventure, were prominent in most decades but less so in others.

Drama and Comedy, however, were consistently dominant across most decades. 

The World genre stood out in three specific decades: the 1960s, 1970s, and 2000s. Its prominence in the first two may be attributed to the Cold War, while in the 2000s, it could be due to globalization.

We can also observe the emergence of Action/Adventure during 1960 and Thriller during 1970.

In [ ]:
all_genre_distr = df_expanded.groupby(['Grouped_genres']).size().reset_index(name="Count")
all_genre_distr = all_genre_distr.sort_values(by=['Count'], ascending=False)

In [ ]:
plot_top_genres_overall(10, all_genre_distr)

### Observations

We see that the two prevalent genres are Drama and Comedy, which supports our previous observations.

## Analysis of movies genres throughout the decades by continent

In [ ]:
# Top 5 genres in each continent for each decade
plot_top_genres_by_continent(df_expanded)

### Observations

Drama is, as expected, the most prevalent genre across each continent. Surprisingly, the Middle East shows only two genres—Short Film and Documentary—in the 2000s, suggesting a limited representation of Middle Eastern films in the dataset.

The appearance of War as a genre in Europe and Oceania during the 1940s correlates with World War II, reflecting how cinema often mirrors historical events during major conflicts.

In North America, the last seven decades show a stable composition of Action/Adventure, Comedy, and Drama, with a touch of Thriller, suggesting a stable trend in popular genres.

Interestingly, in Africa, the most common genres in the recent decade are Documentary and Short Film. This may show a focus on real-life stories on Africa that may resonate with African Audiences.

In [ ]:
# Evolution of the overall top 5 genres in each continent
plot_top_genres_by_continent_overall(df_expanded)

### Observations

A really interesting first observation is the significant drop in the percentage of the Comedy genre in Africa, North America and Europe, with the biggest one happening in Europe. We observe that a really big part of movies before 1900 are comedies, but only less than 20% are comedies from the 1910 decade.

Among the overall top five genres, Drama stands out as the dominant genre across all continents, with each producing more Drama movies than any other genre. However, the distribution of the overall top five genres varies considerably between continents: the distribution seems more balanced in North America (every genre in the overall top 5 takes between 10% and 30% of the total number of movies of these genres) compared to Africa, which has more than 50% of their movies in the Drama genre, leaving less than half distributed over the four other genres.

Other genres do not seem to have a strong fluctuation in their distribution across the decades. We just note that Africa also has a strong decline in the Romance genre between 1930 and 1960, to produce more Drama movies. A guess that could explain this trend is the historical background of Africa during these decades (colonization, global conflicts...).

## Conclusion on the movies genres analysis

From the observations of the evolution of movies genres throughout the decades, there seems to be a hint that societal themes and topics are related to the movies